Analyse the credit card and account statement data

# 1. Preparation

In [ ]:
import pandas as pd
import numpy as np
import time
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.colors as mc
%matplotlib notebook

In [ ]:
data_cc = pd.read_pickle("cc_data_annotated.pkl")
data_account = pd.read_pickle("statement_data_annotated.pkl")
data = data_cc.append(data_account)
data = data.sort_values("Date").reset_index(drop=True)

In [ ]:
first_day = min(data.Date)
last_day = max(data.Date)

In [ ]:
# Reverse "Amount" to get expenses as positive values
data.Amount = -data.Amount

In [ ]:
data

Syntax for removing an entry, if desired:

In [ ]:
# data.loc[data.Subject.apply(lambda x: "CANYON" in x), "Amount"] = 0

Add an expense of 0 to each category for each day, so that the plots make more sense

In [ ]:
cat_names = [
    "EssenTrinken", "Reisen", "Transport", "Freizeit", "Auszahlung",
    "Anschaffungen", "Laufende_Kosten", "Admin", "Investing", "Miete",
    "Gehalt", "Other", "Unassigned"
]

In [ ]:
all_dates = pd.date_range(first_day, last_day)
dates = [date for date in all_dates for n in cat_names[:-1]]
categories = [cat for date in all_dates for cat in cat_names[:-1]]
amounts = np.repeat(0, len(dates))
subjects = np.repeat("-", len(dates))
empty_df = pd.DataFrame({
    "Date": dates,
    "Subject": subjects,
    "Amount": amounts,
    "Category": categories
})
data = data.append(empty_df)
data = data.sort_values("Date")
data.reset_index(drop=True, inplace=True)

In [ ]:
data

# 2. The actual analysis

Calculate 1-month rolling sums

In [ ]:
dfs = []
for cat in cat_names[:-1]:
    cat_data = data[data.Category == cat]
    cat_data = cat_data.drop(columns=["Subject", "Category"]).set_index("Date")
    monthly_cat_data = cat_data.rolling("30D").sum()
    dfs += [monthly_cat_data]

In [ ]:
for i, df in enumerate(dfs):
    df.plot(title=cat_names[i])

Define functions for further analysis

In [ ]:
# Calculates the number of months between start and end day
def calc_num_months(start, end):
    num_months = (end.year - start.year) * 12 + (
        end.month - start.month) + (end.day - start.day) / 30.45
    return num_months


# Calculates monthly averages between two points in time, as well as total expenses
# based on categories_to_sum. Category_values can be floats instead of None, and are then added
# to the monthly average of the respective category (in case there were expenses which are not
# in the data)
def calc_averages(min_time,
                  max_time,
                  categories_to_sum=[0, 1, 2, 3, 4, 5, 6, 8, 9, 11],
                  category_values=[None] * 13):
    per_month_vals = []
    for j, cat in enumerate(cat_names):
        total = sum(data[(data.Category == cat) & (data.Date >= min_time) &
                         (data.Date <= max_time)].Amount)
        per_month = total / calc_num_months(min_time, max_time)
        if category_values[j] is not None:
            per_month += category_values[j]
        per_month_vals += [per_month]
    total_expenses = sum(per_month_vals[i] for i in categories_to_sum
                         if per_month_vals[i] > 0)
    return {
        "total": round(total_expenses, 2),
        "by_category": [round(v, 2) for v in per_month_vals]
    }

# Convert string of format "01.01.20" to datetime object
def to_dt(string):
    return dt.datetime.strptime(string, "%d.%m.%y")

In [ ]:
cat_names

Split up the data into life stages, or episodes, so that costs and expenses can be compared between them:

In [ ]:
life_stages_start_dates = [
    "01.08.16", "28.02.17", "30.03.18", "20.08.18", "28.02.20"
]
life_stages_end_dates = [
    "24.01.17", "30.03.18", "20.08.18", "30.05.20", "30.05.20"
]
life_stages_names = [
    "Living in city A", "Living in city B", "Living in city C",
    "Licing in city D", "City D, last 3 months"
]
rent_city_c = 300
life_stage_averages = [
    [None] * 13, [None] * 9 + [290] + [None] * 3,
    [None] * 6 + [8] + [None] * 6,
    [None] * 6 + [8] + [None] * 2 + [rent_city_c] + [None] * 3,
    [None] * 6 + [8] + [None] * 2 + [rent_city_c] + [None] * 3
]
life_stage_df = pd.DataFrame(
    data={
        "Name": life_stages_names,
        "start": life_stages_start_dates,
        "end": life_stages_end_dates,
        "cat_values": life_stage_averages
    })
life_stage_df

Define the subset of the categories that should count into expenses:

In [ ]:
expense_categories = [0, 1, 2, 3, 4, 5, 6, 8, 9, 11]
expense_names = [cat_names[i] for i in expense_categories]
expense_names

And plot pie charts for each life stage:

In [ ]:
for i in range(len(life_stage_df)):
    avg_dict = calc_averages(to_dt(life_stage_df.start[i]),
                             to_dt(life_stage_df.end[i]),
                             categories_to_sum=expense_categories,
                             category_values=life_stage_df.cat_values[i])
    vals = [
        avg_dict["by_category"][i] for i in expense_categories
        if avg_dict["by_category"][i] > 0
    ]
    labels = [
        cat_names[i] for i in expense_categories
        if avg_dict["by_category"][i] > 0
    ]
    colors = [
        list(mc.XKCD_COLORS)[i + 40] for i in expense_categories
        if avg_dict["by_category"][i] > 0
    ]
    fig1, ax1 = plt.subplots()
    ax1.pie(
        vals,
        labels=labels,
        colors=colors,
        autopct=lambda p: '{:.2f}% ({:,.0f}€)'.format(p,
                                                      p * sum(vals) / 100),
        pctdistance=0.8)
    ax1.set_title(str(life_stage_df.Name[i]), pad=20)
    ax1.text(0, -0.1, "Ausgaben", ha="center", va="center")
    ax1.text(0, -0.2, str(avg_dict["total"]) + "€", ha="center", va="center")
    ax1.text(0, 0.2, "Einnahmen", ha="center", va="center")
    ax1.text(0,
             0.1,
             str(-avg_dict["by_category"][10]) + "€",
             ha="center",
             va="center")
    circle = plt.Circle((0, 0), 0.6, fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(circle)
    ax1.axis('equal')
    plt.tight_layout()
    plt.show()

Code for showing entries of a certain category in a specified timeframe:

In [ ]:
data[(data.Category == "Other") & (data.Amount != 0)
     & (data.Date >= to_dt(life_stage_df.start[2])) &
     (data.Date <= to_dt(life_stage_df.end[2]))]